### Generating the MnAu2 cell structure inputs for the QE calculations

Following the steps in the bcc-fe-exchange-model notebook, read in an initial .cif file (6-atoms) and 
combine with magnetic patterns. Translates in the x,y direction to increase the number of Mn atoms to allow 
for more magnetic patterns. Configuration files saved to working folder.

- 6 atom configuration allows for NM, FM and AFM-A.
- 54 atom configuration

In [17]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D

from ruamel.yaml import YAML
yaml = YAML()

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

### 6 atom:

In [145]:
cif_filepath = "mnau2-6.cif"
structure_mnau26 = from_file(structure_file=cif_filepath)
print(structure_mnau26)

Full Formula (Mn2 Au4)
Reduced Formula: MnAu2
abc   :   3.394839   3.394839   8.880636
angles:  90.000000  90.000000  90.000000
Sites (6)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Mn    0    0    0
  1  Au    0.5  0.5  0.840319
  2  Au    0.5  0.5  0.159681
  3  Mn    0.5  0.5  0.5
  4  Au    0    0    0.340319
  5  Au    0    0    0.659681


In [146]:
pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}

magnetic_patterns = yaml.load(Path("magnetic_patterns-6.yml"))
#pd.DataFrame(magnetic_patterns["6atoms"]).style \
#    .set_table_styles(html_table_style) \
#    .hide_index()

In [147]:
y={}
y['nm'] = magnetic_patterns['6atoms']['nm']
y['f-type'] = magnetic_patterns['6atoms']['f-type']
y['a-type'] = magnetic_patterns['6atoms']['a-type']
new6 = structure_mnau26.copy(y)
new6.make_supercell([5,5,5])
xyz = new6.cart_coords
supercell = pd.concat([pd.DataFrame(new6.species, columns=["species"]),
                          pd.DataFrame(new6.cart_coords, columns=["x","y","z"]), 
                          pd.DataFrame(new6.site_properties["nm"], columns=["nm"]) , 
                          pd.DataFrame(new6.site_properties["f-type"], columns=["f-type"]) ,
                          pd.DataFrame(new6.site_properties["a-type"], columns=["a-type"]) ,
                      ],
                          axis=1, )

supercell.to_csv('mnau2-6.xyz', header=True, index=False, sep='\t')

### 54 Atom: 

In [148]:
structure_mnau254 = structure_mnau26.copy()
structure_mnau254.make_supercell([3,3,1])
structure_mnau254.to(filename="mnau2.cif")
print(structure_mnau254)

Full Formula (Mn18 Au36)
Reduced Formula: MnAu2
abc   :  10.184516  10.184516   8.880636
angles:  90.000000  90.000000  90.000000
Sites (54)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Mn    0         0         0
  1  Mn    0         0.333333  0
  2  Mn    0         0.666667  0
  3  Mn    0.333333  0         0
  4  Mn    0.333333  0.333333  0
  5  Mn    0.333333  0.666667  0
  6  Mn    0.666667  0         0
  7  Mn    0.666667  0.333333  0
  8  Mn    0.666667  0.666667  0
  9  Au    0.166667  0.166667  0.840319
 10  Au    0.166667  0.5       0.840319
 11  Au    0.166667  0.833333  0.840319
 12  Au    0.5       0.166667  0.840319
 13  Au    0.5       0.5       0.840319
 14  Au    0.5       0.833333  0.840319
 15  Au    0.833333  0.166667  0.840319
 16  Au    0.833333  0.5       0.840319
 17  Au    0.833333  0.833333  0.840319
 18  Au    0.166667  0.166667  0.159681
 19  Au    0.166667  0.5       0.159681
 20  Au    0.166667  0.833333  0.159681
 2

In [155]:
magnetic_patterns = yaml.load(Path("magnetic_patterns.yml"))
pd.DataFrame(magnetic_patterns["54atoms"]).style \
    .set_table_styles(html_table_style) \
    .hide_index()


nm,fm,g1,g2,g3,g4,g5
0,1,1,1,1,1,1
0,1,1,1,-1,-1,1
0,1,1,1,1,1,1
0,1,1,-1,1,-1,-1
0,1,1,-1,-1,1,-1
0,1,1,-1,1,-1,-1
0,1,1,1,1,1,1
0,1,1,1,-1,-1,1
0,1,1,1,1,1,1
0,0,0,0,0,0,0


In [156]:
x={}
x['nm'] = magnetic_patterns['54atoms']['nm']
x['fm-type'] = magnetic_patterns['54atoms']['fm']
x['g1-type'] = magnetic_patterns['54atoms']['g1']
x['g2-type'] = magnetic_patterns['54atoms']['g2']
x['g3-type'] = magnetic_patterns['54atoms']['g3']
x['g4-type'] = magnetic_patterns['54atoms']['g4']
x['g5-type'] = magnetic_patterns['54atoms']['g5']
new = structure_mnau254.copy(x)
new.make_supercell([4,4,4])
xyz = pd.concat([pd.DataFrame(new.species, columns=["species"]),
                          pd.DataFrame(new.cart_coords, columns=["x","y","z"]), 
                          pd.DataFrame(new.site_properties["nm"], columns=["nm"]) , 
                          pd.DataFrame(new.site_properties["fm-type"], columns=["fm-type"]) ,
                          pd.DataFrame(new.site_properties["g1-type"], columns=["g1-type"]) ,
                          pd.DataFrame(new.site_properties["g2-type"], columns=["g2-type"]) ,
                          pd.DataFrame(new.site_properties["g3-type"], columns=["g3-type"]) ,
                          pd.DataFrame(new.site_properties["g4-type"], columns=["g4-type"]) ,
                          pd.DataFrame(new.site_properties["g5-type"], columns=["g5-type"]) ,
                          #pd.DataFrame(new.site_properties["g-type"], columns=["g-type"]) 
                      ],
                          axis=1, )
xyz.to_csv('mnau2.xyz', header=True, index=False, sep='\t')

### Calculated energies for different patterns using QE:

In [158]:
qe_energies = pd.read_csv('energy.out', header=None)
#qe_energies.columns = ["type", "count", "total_energy", "energy_per_atom", "relative_energy"]

#qe_energies

### Getting the neighbor information to calculate the exchange parameters:

In [157]:
kB = 1000 * Boltzmann / electron_volt
#mnau2_neighbor_data12 = count_neighbors(cell_structure=structure_mnau212, r=6)
#exchange_model12 = build_model(magnetic_patterns=magnetic_patterns12["12atoms"], neighbor_data=mnau2_neighbor_data12)
#exchange_model12